In [10]:
# Final resize before passing to Yolo 
# Images are landscape at this stage before being processed here 
# The goal is to convert all images to a height and width that is divisible by 32 and a target target_width = 1920
# target_height = 1088 are passed which results in padding the top and bottom evenly by 4 pixels each only. 
# This is for individual folders and was set up as a test before moving to the one that handles folders of folders. 
# Copy any images already at target dimensions directly to the output folder, maintain original format 
# Copy annotations directly without adjustment.
# Pad landscape iamges that have a smaller height 
# Save all resized or padded images in the same format as the original file 
# Only adjust annotations for images that need to be padded or resized. 
# Adjust annotations as needed, applying the padding offset for the y_center.

# Final resize before passing to Yolo 
# Images are landscape at this stage before being processed here 

import os
import cv2

def resize_and_pad_even(input_folder, output_folder, target_width, target_height):
    """
    Resize landscape images to a target width and pad height to reach the target height.
    Handles both .jpg and .png files.
    Ensures images are 1920 × 1080 before processing.
    """
    print("Starting resizing and padding for landscape images...")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, file_name)
            file_extension = os.path.splitext(file_name)[1].lower()
            output_path = os.path.join(output_folder, file_name)

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading {input_path}. Skipping.")
                continue

            original_height, original_width = image.shape[:2]

            # If the image is already at target dimensions, copy it
            if original_width == target_width and original_height == target_height:
                print(f"Image {file_name} already at target dimensions. Copying to output folder.")
                cv2.imwrite(output_path, image)  # Copy the image directly in its original format
                continue

            # Resize the image to target width, keeping aspect ratio
            scale_ratio = target_width / original_width
            new_height = int(original_height * scale_ratio)
            resized_image = cv2.resize(image, (target_width, new_height))

            # Add padding to top and bottom to reach target height
            padding_needed = target_height - new_height
            if padding_needed > 0:
                top_padding = padding_needed // 2
                bottom_padding = padding_needed - top_padding
                padded_image = cv2.copyMakeBorder(
                    resized_image, top_padding, bottom_padding, 0, 0,
                    borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0]  # Black padding
                )
            else:
                padded_image = resized_image

            # Save the processed image in the original file format
            cv2.imwrite(output_path, padded_image)

    print("Completed resizing and padding for landscape images.")


def adjust_annotations_for_folder_landscape(input_annotation_folder, input_image_folder, output_annotation_folder, target_width, target_height):
    """
    Adjust YOLO annotations for all landscape images in a folder and save them for a batch test case.
    Handles both .jpg and .png files.
    Ensures images are 1920 × 1080 before processing annotations.
    """
    # Ensure output folder exists
    if not os.path.exists(output_annotation_folder):
        os.makedirs(output_annotation_folder)

    # Process each annotation file
    for annotation_file in os.listdir(input_annotation_folder):
        if annotation_file.endswith('.txt'):
            annotation_path = os.path.join(input_annotation_folder, annotation_file)
            
            # Look for both .jpg and .png images
            image_path_jpg = os.path.join(input_image_folder, annotation_file.replace('.txt', '.jpg'))
            image_path_png = os.path.join(input_image_folder, annotation_file.replace('.txt', '.png'))
            image_path = image_path_jpg if os.path.exists(image_path_jpg) else image_path_png

            output_annotation_path = os.path.join(output_annotation_folder, annotation_file)

            # Check if the corresponding image exists
            if not os.path.exists(image_path):
                print(f"Image file not found for annotation: {annotation_file}. Skipping.")
                continue

            # Get image dimensions
            image = cv2.imread(image_path)
            original_height, original_width = image.shape[:2]

            # If the image is already at target dimensions, copy annotation directly
            if original_width == target_width and original_height == target_height:
                print(f"Annotation for {annotation_file} corresponds to an image already at target dimensions. Copying directly.")
                with open(annotation_path, 'r') as file:
                    annotations = file.readlines()
                with open(output_annotation_path, 'w') as file:
                    file.writelines(annotations)
                continue

            # Calculate padding offset
            padding_needed = target_height - original_height
            top_padding = padding_needed // 2
            print(f"Processing {annotation_file}: top_padding={top_padding}")

            adjusted_lines = []

            # Read annotations
            with open(annotation_path, 'r') as file:
                annotations = file.readlines()

            for line in annotations:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())

                # Scale coordinates and dimensions
                x_center_scaled = x_center * original_width
                y_center_scaled = y_center * original_height
                width_scaled = width * original_width
                height_scaled = height * original_height

                # Reverse padding offset (move y_center back into padded space)
                y_center_scaled += top_padding

                # Clamp to image bounds
                x_left = max(0, x_center_scaled - width_scaled / 2)
                x_right = min(target_width, x_center_scaled + width_scaled / 2)
                y_top = max(0, y_center_scaled - height_scaled / 2)
                y_bottom = min(target_height, y_center_scaled + height_scaled / 2)

                # Recalculate dimensions
                width_clamped = max(0, x_right - x_left)
                height_clamped = max(0, y_bottom - y_top)

                if width_clamped > 0 and height_clamped > 0:
                    x_center_normalized = (x_left + x_right) / (2 * target_width)
                    y_center_normalized = (y_top + y_bottom) / (2 * target_height)
                    width_normalized = width_clamped / target_width
                    height_normalized = height_clamped / target_height

                    # Append corrected annotation
                    adjusted_lines.append(
                        f"{int(class_id)} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"
                    )
                else:
                    print(f"Bounding box for class {class_id} is invalid after clamping: skipping.")

            # Save adjusted annotations
            with open(output_annotation_path, 'w') as file:
                file.write('\n'.join(adjusted_lines))


###################################################################################################################

# Paths and target dimensions
#input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151"
#output_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_151FINAL"
input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_155"
output_folder = "D:/FlagDetectionDatasets/ExportedDatasetsSelected/Job_155FINAL"
target_width = 1920
target_height = 1088

# Resize and PAD landscape images in the source folder - WILL PAD BY 4 PX AT TOP AND BOTTOM 
resize_and_pad_even(input_folder, output_folder, target_width, target_height)
print("Starting resizing ...")

# Update annotations for all images in a a folader
adjust_annotations_for_folder_landscape(input_folder, input_folder, output_folder, target_width, target_height)
print("Starting updating annotations ...")



Starting resizing and padding for landscape images...
Completed resizing and padding for landscape images.
Starting resizing ...
Processing Job_155_000019.txt: top_padding=4
Processing Job_155_000032.txt: top_padding=4
Processing Job_155_000028.txt: top_padding=4
Processing Job_155_000033.txt: top_padding=4
Processing Job_155_000025.txt: top_padding=4
Processing Job_155_000030.txt: top_padding=4
Processing Job_155_000040.txt: top_padding=4
Processing Job_155_000006.txt: top_padding=4
Processing Job_155_000010.txt: top_padding=4
Processing Job_155_000045.txt: top_padding=4
Processing Job_155_000017.txt: top_padding=4
Processing Job_155_000009.txt: top_padding=4
Processing Job_155_000008.txt: top_padding=4
Processing Job_155_000014.txt: top_padding=4
Processing Job_155_000022.txt: top_padding=4
Processing Job_155_000013.txt: top_padding=4
Processing Job_155_000043.txt: top_padding=4
Processing Job_155_000041.txt: top_padding=4
Processing Job_155_000052.txt: top_padding=4
Processing Job_1